# Нейронные сети
__Суммарное количество баллов: 10__

В этом домашнем задании можно использовать готовые классы библиотек PyTorch, Keras, TensorFlow.

### Задание 1 (4 балла)

Создайте сеть из трех сверточных слоев по 8 сверток 3х3 и двух полносвязных слоев по 64 нейрона. 

Обучите сеть на датасете mnist с тремя разными функциями активации в слоях (sigmoid, tanh, ReLU).

In [57]:
import pathlib
import string
import keras
import pandas as pd
import numpy as np

from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [58]:
def read_mnist_dataset(path_to_csv):
    # Возвращает пару из X и y. X - массив векторов. y - соответствующие векторам метки
    df = shuffle(pd.read_csv(path_to_csv))
    y = df['label'].values
    X = df.drop('label', axis=1).values
    return X, y

def train_test_split(X, y, ratio=0.8):
    n = int(len(y) * ratio)
    return X[:n], y[:n], X[n:], y[n:]

In [59]:
batch_size = 128
num_classes = 10
epochs = 5
img_rows, img_cols = 28, 28

X, y = read_mnist_dataset('hw1/mnist.csv')

X_train, y_train, X_test, y_test = train_test_split(X, y)
print(X_train.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


(8000, 784)


In [65]:
def activation_mode(activation, optimizer):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation=activation,
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation=activation))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test))
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

### ReLU

In [61]:
activation_mode('relu', keras.optimizers.Adadelta())

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 13s 2ms/step - loss: 0.8142 - accuracy: 0.7398 - val_loss: 0.2453 - val_accuracy: 0.9245
Epoch 2/5
8000/8000 [==============================] - 11s 1ms/step - loss: 0.2454 - accuracy: 0.9266 - val_loss: 0.1713 - val_accuracy: 0.9450
Epoch 3/5
8000/8000 [==============================] - 12s 1ms/step - loss: 0.1644 - accuracy: 0.9506 - val_loss: 0.1154 - val_accuracy: 0.9660
Epoch 4/5
8000/8000 [==============================] - 12s 1ms/step - loss: 0.1154 - accuracy: 0.9635 - val_loss: 0.1020 - val_accuracy: 0.9715
Epoch 5/5
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0979 - accuracy: 0.9693 - val_loss: 0.0977 - val_accuracy: 0.9700
Test loss: 0.09766330996900797
Test accuracy: 0.9700000286102295


### Sigmoid

In [66]:
activation_mode('sigmoid', 'adam')

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 19s 2ms/step - loss: 2.6981 - accuracy: 0.0997 - val_loss: 2.3189 - val_accuracy: 0.1065
Epoch 2/5
8000/8000 [==============================] - 20s 2ms/step - loss: 2.4553 - accuracy: 0.1004 - val_loss: 2.3033 - val_accuracy: 0.1130
Epoch 3/5
8000/8000 [==============================] - 19s 2ms/step - loss: 2.4277 - accuracy: 0.0984 - val_loss: 2.3069 - val_accuracy: 0.1130
Epoch 4/5
8000/8000 [==============================] - 19s 2ms/step - loss: 2.3880 - accuracy: 0.1034 - val_loss: 2.3055 - val_accuracy: 0.1130
Epoch 5/5
8000/8000 [==============================] - 19s 2ms/step - loss: 2.3730 - accuracy: 0.1021 - val_loss: 2.3088 - val_accuracy: 0.0920
Test loss: 2.3087642860412596
Test accuracy: 0.09200000017881393


### Tanh

In [63]:
activation_mode('tanh', keras.optimizers.Adadelta())

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 21s 3ms/step - loss: 0.5773 - accuracy: 0.8200 - val_loss: 0.2599 - val_accuracy: 0.9215
Epoch 2/5
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2391 - accuracy: 0.9291 - val_loss: 0.1827 - val_accuracy: 0.9425
Epoch 3/5
8000/8000 [==============================] - 20s 3ms/step - loss: 0.1707 - accuracy: 0.9480 - val_loss: 0.1538 - val_accuracy: 0.9545
Epoch 4/5
8000/8000 [==============================] - 20s 2ms/step - loss: 0.1344 - accuracy: 0.9561 - val_loss: 0.1333 - val_accuracy: 0.9635
Epoch 5/5
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0973 - accuracy: 0.9715 - val_loss: 0.1282 - val_accuracy: 0.9605
Test loss: 0.12818317437171936
Test accuracy: 0.9605000019073486


### Задание 2 (6 баллов)

Достигните точности 87% на test датасете notMNIST.

Архитектура сети может быть любая. Можно использовать Data Augmentation.

In [52]:
def read_not_mnist(letter2label):
    image_arrays = []
    labels = []
    
    dataset_path = pathlib.Path('hw1/notMNIST_small')
    for letter in letter2label:
        for image_path in (dataset_path / letter).glob('**/*.png'):
            img = load_img(str(image_path))
            x = img_to_array(img)
            image_arrays.append(x)
            labels.append(letter2label[letter])
    X = np.array(image_arrays)
    y = np.array(labels)
    # shuffle
    n = y.size
    idx = np.random.choice(n, n, replace=False)
    return X[idx], y[idx]

In [53]:
batch_size = 128
num_classes = 10
epochs = 5
img_rows, img_cols = 28, 28

letters = list(string.ascii_uppercase[:10])
letter2label = dict(zip(letters, range(10)))
label2letter = dict(zip(range(10), letters))

X, y = read_not_mnist(letter2label)
X_train, y_train, X_test, y_test = train_test_split(X, y)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
input_shape = X_train.shape[1:]
print(input_shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

(28, 28, 3)


In [54]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 14979 samples, validate on 3745 samples
Epoch 1/5
14979/14979 [==============================] - 23s 2ms/step - loss: 0.7355 - accuracy: 0.7739 - val_loss: 0.3227 - val_accuracy: 0.9095
Epoch 2/5
14979/14979 [==============================] - 21s 1ms/step - loss: 0.3348 - accuracy: 0.8994 - val_loss: 0.2359 - val_accuracy: 0.9279
Epoch 3/5
14979/14979 [==============================] - 21s 1ms/step - loss: 0.2660 - accuracy: 0.9205 - val_loss: 0.2374 - val_accuracy: 0.9332
Epoch 4/5
14979/14979 [==============================] - 21s 1ms/step - loss: 0.2260 - accuracy: 0.9290 - val_loss: 0.2484 - val_accuracy: 0.9284
Epoch 5/5
14979/14979 [==============================] - 21s 1ms/step - loss: 0.2018 - accuracy: 0.9382 - val_loss: 0.2074 - val_accuracy: 0.9431
Test loss: 0.2073681864904262
Test accuracy: 0.9431241750717163
